# Specify Paths 

In [ ]:

fpga_v3_data_collection_path = "/home/ndrakes1/force_estimation/preprocess/fpgav3-data-collection/" # SET THIS ONCE
preproces_path = "/home/ndrakes1/force_estimation/preprocess/force-estimation-fpga-data-preprocess" # SET THIS ONCE

true_preprocess_path = "~/force_estimation/preprocess/"


unit_convert_path = fpga_v3_data_collection_path + "unit_convert/unit_convert.py"
si_unit_json_path = "/home/ndrakes1/force_estimation/data/XML_FILES/sawRobotIO1394-PSM1-292409.xml.json"
unit_convert_cmd = "python3 " + unit_convert_path + " -c " + si_unit_json_path + " -f " 

train_name_path_csv = "/home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/train.csv" # SET THIS
val_test_name_path_csv = "/home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/val.csv" # SET THIS


train_name_path_unit_convert_csv = "/home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/train_unitConvert.csv" # SET THIS
val_ttest_name_path_unit_convert_csv = "/home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/val_unitConvert.csv" # SET THIS

path_to_data_foleder = "/home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min"
new_preprocessed_data_folder = "/preprocessed"

unit_convert_train_cmd = unit_convert_cmd + train_name_path_csv
unit_convert_val_test_cmd = unit_convert_cmd + val_test_name_path_csv

ORIGINAL_FREQ = 10000

CUTOFF = True
CUTOFF_SECS = 20

if CUTOFF == True:
    new_preprocessed_data_folder += "_CUTOFF_" + str(CUTOFF_SECS)

INTERPOLATE = True

if INTERPOLATE == True:
    new_preprocessed_data_folder += "_INTERPOLATE" 


FILTER = True
FILTER_FREQ = 200
FILTER_VELOCITY = True
FILTER_POSITION = True

if FILTER == True:
    new_preprocessed_data_folder += "_FILTER_" + str(FILTER_FREQ)

DOWNSAMPLE = True
DOWNSAMPLE_FREQ = 1000
DOWNSAMPLE_MOVING_AVERAGE = True

if DOWNSAMPLE == True:
    new_preprocessed_data_folder += "_DOWNSAMPLE_" + str(DOWNSAMPLE_FREQ)



In [29]:
## making preprocessed data dir 
import subprocess

new_dir = path_to_data_foleder + new_preprocessed_data_folder
train_sub = new_dir + "/train/joints/"
val_sub = new_dir + "/val/joints/"
test_sub = new_dir + "/test/joints/"
mk_new_dir_cmd = "mkdir -p " + train_sub + " " + val_sub + " " + test_sub

subprocess.run(mk_new_dir_cmd, shell=True)

CompletedProcess(args='mkdir -p /home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/preprocessed_CUTOFF_20_INTERPOLATE_FILTER_200/train/joints/ /home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/preprocessed_CUTOFF_20_INTERPOLATE_FILTER_200/val/joints/ /home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/preprocessed_CUTOFF_20_INTERPOLATE_FILTER_200/test/joints/', returncode=0)

## paths for high freq preprocessing

In [21]:

preprocessing_script_path = preproces_path + "/preprocessing.py"
split_val_test_path = preproces_path + "/split_val_test.py"

preprocessing_train_cmd = "python3 " + preprocessing_script_path + " " + train_name_path_unit_convert_csv + " " + train_sub + "interpolated_all_joints.csv"
preprocessing_val_test_cmd ="python3 " + preprocessing_script_path + " " + val_ttest_name_path_unit_convert_csv + " " + val_sub + "interpolated_all_joints.csv"

print(preprocessing_train_cmd)

python3 /home/ndrakes1/force_estimation/preprocess/force-estimation-fpga-data-preprocess/preprocessing.py /home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/train_unitConvert.csv /home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/preprocessed_CUTOFF_20_INTERPOLATE_FILTER_200/train/joints/interpolated_all_joints.csv


# Unit Conversion to SI units

In [ ]:

subprocess.run(unit_convert_val_test_cmd, shell=True)
subprocess.run(unit_convert_train_cmd, shell=True)

Data written to /home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/val_unitConvert.csv
Data written to /home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/train_unitConvert.csv


CompletedProcess(args='python3 /home/ndrakes1/force_estimation/preprocess/fpgav3-data-collection/unit_convert/unit_convert.py -c /home/ndrakes1/force_estimation/data/XML_FILES/sawRobotIO1394-PSM1-292409.xml.json -f /home/ndrakes1/force_estimation/data/capture_8-18-25/10kHz_20min/train.csv', returncode=0)

# High Freq Preprocessing

In [ ]:
# # interpolate train and place it in correct directory
# # train
subprocess.run(preprocessing_train_cmd, shell=True)
print("finished preprocessing train ")

## val test
subprocess.run(preprocessing_val_test_cmd, shell=True)
print("preprocess split and val")

# # split
split_val_test_cmd = "python3 " + preproces_path + "/split_val_test.py " + " " + val_sub + "interpolated_all_joints.csv"
subprocess.run(split_val_test_cmd, shell=True)
print("split into val and test")

!mv {preproces_path}/val.csv {val_sub}interpolated_all_joints.csv
!mv {preproces_path}/test.csv {test_sub}interpolated_all_joints.csv


## Preprocessed Training File Paths

In [38]:
import os
import pandas as pd

train_file = os.path.join(train_sub, "interpolated_all_joints.csv")
val_file = os.path.join(val_sub, "interpolated_all_joints.csv")
test_file = os.path.join(test_sub, "interpolated_all_joints.csv")

train_df = pd.read_csv(train_file, header=None)
val_df =  pd.read_csv(val_file, header=None)
test_df = pd.read_csv(test_file, header=None)

## Cutoff


In [33]:
from cutoff import truncate_dataframe


if (CUTOFF == True):
    ## TRAIN
    FREQ =  ORIGINAL_FREQ if DOWNSAMPLE == False else DOWNSAMPLE_FREQ
    ## TRAIN
    train_df = truncate_dataframe(train_df, CUTOFF_SECS, FREQ)
    val_df = truncate_dataframe(val_df, CUTOFF_SECS, FREQ)
    test_df = truncate_dataframe(test_df, CUTOFF_SECS, FREQ)

total rows:  12111047
total rows:  1845688
total rows:  1845688


## Interpolate

In [34]:
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

if (INTERPOLATE == True):
    
    train_df = interpolate_dataframe_to_sample_rate(train_df, ORIGINAL_FREQ)
    val_df = interpolate_dataframe_to_sample_rate(val_df, ORIGINAL_FREQ)
    test_df = interpolate_dataframe_to_sample_rate(test_df, ORIGINAL_FREQ)


[0.0000000e+00 1.0053000e-04 1.9984800e-04 ... 1.1711044e+03 1.1711045e+03
 1.1711046e+03]
[0.00000000e+00 9.95400000e-05 1.99080000e-04 ... 1.44568499e+02
 1.44568599e+02 1.44568699e+02]
[0.000000e+00 1.001940e-04 1.995180e-04 ... 1.445685e+02 1.445686e+02
 1.445687e+02]


## FIR Filtering

In [ ]:
import importlib
import filter  # regular import first
importlib.reload(filter)



if (FILTER == True):
    # Design filter
    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)

    # Apply to train/val/test CSVs
    train_df = filter.apply_filter_to_torque_feedback_df(train_df, fir_coeffs, filter_velocity=FILTER_VELOCITY)
    val_df = filter.apply_filter_to_torque_feedback_df(val_df, fir_coeffs, filter_velocity=FILTER_VELOCITY)
    test_df = filter.apply_filter_to_torque_feedback_df(test_df, fir_coeffs, filter_velocity=FILTER_VELOCITY)

    print("FILTERED")

## Downsample


In [36]:
from downsample import downsample_dataframe

if (DOWNSAMPLE == True):
    train_df = downsample_dataframe(train_df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
    val_df = downsample_dataframe(val_df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
    test_df = downsample_dataframe(test_df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)


## SAVE TO CSV

In [37]:
train_df.to_csv(train_file, index=False, header=False)
val_df.to_csv(val_file, index=False, header=False)
test_df.to_csv(test_file, index=False, header=False)

KeyboardInterrupt: 